### DATA 622 / Homework 5

#### November 2018 / Ilya Kats

This Notebook follows DataCamp's Apache Spark Tutorial available at https://www.datacamp.com/community/tutorials/apache-spark-tutorial-machine-learning. Almost all code is from the tutorial. For training purposes, this page includes code to build a decision tree model in addition to the linear regression model. The code for decision tree model was modified from Susan Li's post on Towards Data Science (https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a). 

##### Setup and Data Connection

Open Spark session using `pyspark`.

In [1]:
# Find Spark
import findspark
findspark.init()

# Import SparkSession
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext

##### Data Import and Exploration

Import data into RDDs.

In [2]:
root_path = 'C:\\Users\\ikats\\OneDrive\\Documents\\GitHub\\CUNY-DATA622\\HW5\\'

# Load in the data
rdd = sc.textFile(root_path+'cal_housing.data')

# Load in the header
header = sc.textFile(root_path+'cal_housing.domain')

In [3]:
# Read header information
header.collect()

['longitude: continuous.',
 'latitude: continuous.',
 'housingMedianAge: continuous. ',
 'totalRooms: continuous. ',
 'totalBedrooms: continuous. ',
 'population: continuous. ',
 'households: continuous. ',
 'medianIncome: continuous. ',
 'medianHouseValue: continuous. ']

In [4]:
# Grab the first 5 records of the data set
rdd.take(5)

['-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000',
 '-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000',
 '-122.240000,37.850000,52.000000,1467.000000,190.000000,496.000000,177.000000,7.257400,352100.000000',
 '-122.250000,37.850000,52.000000,1274.000000,235.000000,558.000000,219.000000,5.643100,341300.000000',
 '-122.250000,37.850000,52.000000,1627.000000,280.000000,565.000000,259.000000,3.846200,342200.000000']

In [5]:
# Split lines on commas and inspect first 2 entries
rdd = rdd.map(lambda line: line.split(","))
rdd.take(2)

[['-122.230000',
  '37.880000',
  '41.000000',
  '880.000000',
  '129.000000',
  '322.000000',
  '126.000000',
  '8.325200',
  '452600.000000'],
 ['-122.220000',
  '37.860000',
  '21.000000',
  '7099.000000',
  '1106.000000',
  '2401.000000',
  '1138.000000',
  '8.301400',
  '358500.000000']]

Convert to Dataframe.

In [6]:
# Import the necessary modules 
from pyspark.sql import Row

# Map the RDD to a DF
df = rdd.map(lambda line: Row(longitude=line[0], 
                              latitude=line[1], 
                              housingMedianAge=line[2],
                              totalRooms=line[3],
                              totalBedRooms=line[4],
                              population=line[5], 
                              households=line[6],
                              medianIncome=line[7],
                              medianHouseValue=line[8])).toDF()

In [7]:
# Show the top 20 rows 
df.show()

+-----------+----------------+---------+-----------+----------------+------------+-----------+-------------+-----------+
| households|housingMedianAge| latitude|  longitude|medianHouseValue|medianIncome| population|totalBedRooms| totalRooms|
+-----------+----------------+---------+-----------+----------------+------------+-----------+-------------+-----------+
| 126.000000|       41.000000|37.880000|-122.230000|   452600.000000|    8.325200| 322.000000|   129.000000| 880.000000|
|1138.000000|       21.000000|37.860000|-122.220000|   358500.000000|    8.301400|2401.000000|  1106.000000|7099.000000|
| 177.000000|       52.000000|37.850000|-122.240000|   352100.000000|    7.257400| 496.000000|   190.000000|1467.000000|
| 219.000000|       52.000000|37.850000|-122.250000|   341300.000000|    5.643100| 558.000000|   235.000000|1274.000000|
| 259.000000|       52.000000|37.850000|-122.250000|   342200.000000|    3.846200| 565.000000|   280.000000|1627.000000|
| 193.000000|       52.000000|37

In [8]:
# Print the schema of `df`
df.printSchema()

root
 |-- households: string (nullable = true)
 |-- housingMedianAge: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- medianHouseValue: string (nullable = true)
 |-- medianIncome: string (nullable = true)
 |-- population: string (nullable = true)
 |-- totalBedRooms: string (nullable = true)
 |-- totalRooms: string (nullable = true)



In [10]:
# Convert to proper data types
from pyspark.sql.types import *

df = df.withColumn("longitude", df["longitude"].cast(FloatType())) \
   .withColumn("latitude", df["latitude"].cast(FloatType())) \
   .withColumn("housingMedianAge",df["housingMedianAge"].cast(FloatType())) \
   .withColumn("totalRooms", df["totalRooms"].cast(FloatType())) \
   .withColumn("totalBedRooms", df["totalBedRooms"].cast(FloatType())) \
   .withColumn("population", df["population"].cast(FloatType())) \
   .withColumn("households", df["households"].cast(FloatType())) \
   .withColumn("medianIncome", df["medianIncome"].cast(FloatType())) \
   .withColumn("medianHouseValue", df["medianHouseValue"].cast(FloatType()))

In [11]:
df.printSchema()

root
 |-- households: float (nullable = true)
 |-- housingMedianAge: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- medianHouseValue: float (nullable = true)
 |-- medianIncome: float (nullable = true)
 |-- population: float (nullable = true)
 |-- totalBedRooms: float (nullable = true)
 |-- totalRooms: float (nullable = true)



Alternative way of converting column data types.

In [12]:
# Import all from `sql.types`
# from pyspark.sql.types import *

# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
  for name in names: 
     df = df.withColumn(name, df[name].cast(newType))
  return df 

# Assign all column names to `columns`
columns = ['households', 'housingMedianAge', 'latitude', 'longitude', 'medianHouseValue', 'medianIncome', 'population', 'totalBedRooms', 'totalRooms']

# Conver the `df` columns to `FloatType()`
# df = convertColumn(df, columns, FloatType())

In [13]:
# Selecting specific columns and viewing first 10 observations
df.select('population','totalBedRooms').show(10)

+----------+-------------+
|population|totalBedRooms|
+----------+-------------+
|     322.0|        129.0|
|    2401.0|       1106.0|
|     496.0|        190.0|
|     558.0|        235.0|
|     565.0|        280.0|
|     413.0|        213.0|
|    1094.0|        489.0|
|    1157.0|        687.0|
|    1206.0|        665.0|
|    1551.0|        707.0|
+----------+-------------+
only showing top 10 rows



In [14]:
# Top 20 median ages and number of houses by age
df.groupBy("housingMedianAge").count().sort("housingMedianAge",ascending=False).show()

+----------------+-----+
|housingMedianAge|count|
+----------------+-----+
|            52.0| 1273|
|            51.0|   48|
|            50.0|  136|
|            49.0|  134|
|            48.0|  177|
|            47.0|  198|
|            46.0|  245|
|            45.0|  294|
|            44.0|  356|
|            43.0|  353|
|            42.0|  368|
|            41.0|  296|
|            40.0|  304|
|            39.0|  369|
|            38.0|  394|
|            37.0|  537|
|            36.0|  862|
|            35.0|  824|
|            34.0|  689|
|            33.0|  615|
+----------------+-----+
only showing top 20 rows



In [18]:
# Summary statistics - first 5 columns
df.select(df.columns[0:5]).describe().show()

+-------+------------------+------------------+------------------+-------------------+------------------+
|summary|        households|  housingMedianAge|          latitude|          longitude|  medianHouseValue|
+-------+------------------+------------------+------------------+-------------------+------------------+
|  count|             20640|             20640|             20640|              20640|             20640|
|   mean| 499.5396802325581|28.639486434108527| 35.63186143109965|-119.56970444871473|206855.81690891474|
| stddev|382.32975283161136|12.585557612111613|2.1359523806029554| 2.0035317429328914|115395.61587441381|
|    min|               1.0|               1.0|             32.54|            -124.35|           14999.0|
|    max|            6082.0|              52.0|             41.95|            -114.31|          500001.0|
+-------+------------------+------------------+------------------+-------------------+------------------+



In [19]:
# Summary statistics - remaining columns
df.select(df.columns[5:]).describe().show()

+-------+------------------+------------------+-----------------+------------------+
|summary|      medianIncome|        population|    totalBedRooms|        totalRooms|
+-------+------------------+------------------+-----------------+------------------+
|  count|             20640|             20640|            20640|             20640|
|   mean|3.8706710030346416|1425.4767441860465|537.8980135658915|2635.7630813953488|
| stddev|1.8998217183639672|1132.4621217653385|421.2479059431315|2181.6152515827994|
|    min|            0.4999|               3.0|              1.0|               2.0|
|    max|           15.0001|           35682.0|           6445.0|           39320.0|
+-------+------------------+------------------+-----------------+------------------+



##### Data Preprocessing

Preprocessing target variable - `medianHouseValue` - by experessing it in units of 100,000.

In [20]:
# Import all from `sql.functions` 
from pyspark.sql.functions import *

# Adjust the values of `medianHouseValue`
df = df.withColumn("medianHouseValue", col("medianHouseValue")/100000)

# Show the first 2 lines of `df`
df.take(2)

[Row(households=126.0, housingMedianAge=41.0, latitude=37.880001068115234, longitude=-122.2300033569336, medianHouseValue=4.526, medianIncome=8.325200080871582, population=322.0, totalBedRooms=129.0, totalRooms=880.0),
 Row(households=1138.0, housingMedianAge=21.0, latitude=37.86000061035156, longitude=-122.22000122070312, medianHouseValue=3.585, medianIncome=8.301400184631348, population=2401.0, totalBedRooms=1106.0, totalRooms=7099.0)]

##### Feature Engineering

Add new features: 

- *Rooms per household*, which refers to the number of rooms in households per block group,
- *Population per household*, which basically gives an indication of how many people live in households per block group, and
- *Bedrooms per room*, which gives an idea about how many rooms are bedrooms per block group.

In [21]:
# Divide `totalRooms` by `households`
roomsPerHousehold = df.select(col("totalRooms")/col("households"))

# Divide `population` by `households`
populationPerHousehold = df.select(col("population")/col("households"))

# Divide `totalBedRooms` by `totalRooms`
bedroomsPerRoom = df.select(col("totalBedRooms")/col("totalRooms"))

# Add the new columns to `df`
df = df.withColumn("roomsPerHousehold", col("totalRooms")/col("households")) \
   .withColumn("populationPerHousehold", col("population")/col("households")) \
   .withColumn("bedroomsPerRoom", col("totalBedRooms")/col("totalRooms"))
   
# Inspect the result
df.first()

Row(households=126.0, housingMedianAge=41.0, latitude=37.880001068115234, longitude=-122.2300033569336, medianHouseValue=4.526, medianIncome=8.325200080871582, population=322.0, totalBedRooms=129.0, totalRooms=880.0, roomsPerHousehold=6.984126984126984, populationPerHousehold=2.5555555555555554, bedroomsPerRoom=0.14659090909090908)

##### Standardization

In [22]:
# Re-order and select columns
df = df.select("medianHouseValue", 
              "totalBedRooms", 
              "population", 
              "households", 
              "medianIncome", 
              "roomsPerHousehold", 
              "populationPerHousehold", 
              "bedroomsPerRoom")

Separate target variable from other features to scale.

In [23]:
# Import `DenseVector`
from pyspark.ml.linalg import DenseVector

# Define the `input_data` 
input_data = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# Replace `df` with the new DataFrame
df = spark.createDataFrame(input_data, ["label", "features"])

In [24]:
df.first()

Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 2.5556, 0.1466]))

Scale features using Spark ML library.

In [25]:
# Import `StandardScaler` 
from pyspark.ml.feature import StandardScaler

# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

# Fit the DataFrame to the scaler
scaler = standardScaler.fit(df)

# Transform the data in `df` with the scaler
scaled_df = scaler.transform(df)

# Inspect the result
scaled_df.take(2)

[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 2.5556, 0.1466]), features_scaled=DenseVector([0.3062, 0.2843, 0.3296, 4.3821, 2.8228, 0.2461, 2.5264])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 2.1098, 0.1558]), features_scaled=DenseVector([2.6255, 2.1202, 2.9765, 4.3696, 2.5213, 0.2031, 2.6851]))]

##### Machine Learning Model with Spark ML

Split data into testing and training sets.

In [26]:
# Split the data into train and test sets
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

Initialize the linear regression model and fit data to it. Then generate predictions. Then compare predictions with known labels.

In [27]:
# Import `LinearRegression`
from pyspark.ml.regression import LinearRegression

# Initialize `lr`
lr = LinearRegression(labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the data to the model
linearModel = lr.fit(train_data)

In [31]:
# Generate predictions
predicted = linearModel.transform(test_data)

In [32]:
# Extract the predictions and the "known" correct labels
predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
labels = predicted.select("label").rdd.map(lambda x: x[0])

# Zip `predictions` and `labels` into a list
predictionAndLabel = predictions.zip(labels).collect()

# Print out first 5 instances of `predictionAndLabel` 
predictionAndLabel[:5]

[(1.4491508524918457, 0.14999),
 (1.5705029404692372, 0.14999),
 (2.148727956912464, 0.14999),
 (1.5831547768979277, 0.344),
 (1.5182107797955968, 0.398)]

Evaluate model.

In [34]:
# Coefficients for the model
linearModel.coefficients

DenseVector([0.0, 0.0, 0.0, 0.2762, 0.0, 0.0, 0.0])

In [35]:
# Intercept for the model
linearModel.intercept

0.9903995774620005

In [36]:
# Get the RMSE
linearModel.summary.rootMeanSquaredError

0.8692118678997669

In [37]:
# Get the R2
linearModel.summary.r2

0.4240895287218379

R2 is very low at 42.4%. There is definitely room for improvement.

A bit more exprimenting on getting R2 and RMSE values on test data and my building decision tree model.

In [53]:
# Build decision tree model on the same training data
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ="features", labelCol = "label")
dtModel = dt.fit(train_data)

# Generate prediction using decision tree model
dt_predicted = dtModel.transform(test_data)

In [54]:
# Set up ML Spark's regression evaluator to evaluate test data
from pyspark.ml.evaluation import RegressionEvaluator

rmse_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                     labelCol="label", metricName="rmse")
r2_evaluator = RegressionEvaluator(predictionCol="prediction", \
                                   labelCol="label", metricName="r2")

In [55]:
# R2 and RMSE value of linear regression model
print("Linear Model: R2 on test data = %g" % r2_evaluator.evaluate(predicted))
print("Linear Model: RMSE on test data = %g" % rmse_evaluator.evaluate(predicted))

Linear Model: R2 on test data = 0.399059
Linear Model: RMSE on test data = 0.919953


In [56]:
# R2 and RMSE value of decision tree model
print("Decision Tree Model: R2 on test data = %g" % r2_evaluator.evaluate(dt_predicted))
print("Decision Tree Model: RMSE on test data = %g" % rmse_evaluator.evaluate(dt_predicted))

Decision Tree Model: R2 on test data = 0.602836
Decision Tree Model: RMSE on test data = 0.747885


R2 is noticeably higher and RMSE is noticeably lower for the decision tree model. A lot more should be done to evaluate models, but for quick testing in this example, decision tree seems to be a better model than linear regression model.

##### Stop Spark Session

In [58]:
spark.stop()